## Intro

My friend is learning how to program with pandas. Durring the pandemic, he runs classes online and the output is in and out times for the students. Some have connection issues that make them clock in and out a lot, but they are still in class most of the time. Other students skip class altogether. Can we use pandas to figure out how long each student was in class?


In [1]:
import pandas as pd
import numpy as np

## Setup
First we are going to generate the data. If you are just learning pandas and what I am doing below looks a little intence, don't worry about it. Basically, we are creating 4 students with random in/out times. Then we shuffle the order so that the times and IDS need to be sorted later.

In [2]:
student_log = pd.DataFrame(columns=['student_id', 'timestamp'])

for i in range(4):
    num_pairs = np.random.randint(low=1, high=6)*2
    inout = np.arange(num_pairs, dtype=np.float)
    inout += np.random.uniform(0,30,num_pairs)
    inout.sort()
    student_log = student_log.append(pd.DataFrame({'student_id': i, 'timestamp': inout}), ignore_index=True)

student_log = student_log.sample(frac=1.0).reset_index(drop=True)

<!-- TEASER_END -->

In [3]:
student_log

,student_id,timestamp
0,0,16.304589
1,2,29.953800
2,2,17.304851
3,1,25.151845
4,0,19.970583
5,0,9.599590
6,0,28.629028
7,2,16.248891
8,2,9.836785
9,2,14.812535


## Data Clean

Now that we have the data, we should get it in the proper order. I am going to sort by `student_id` then by `timestamp`. Then we can calcualte how long each student was in class.

In [4]:
student_log = student_log.sort_values(['student_id', 'timestamp'])
student_log

,student_id,timestamp
5,0,9.599590
13,0,13.332287
0,0,16.304589
15,0,18.955793
4,0,19.970583
6,0,28.629028
3,1,25.151845
11,1,25.736422
8,2,9.836785
10,2,13.742524


The index is out of order, but that is OK. You can reset the index if needed but we are going to ignore it for now.

## Aggrigate the data

Now we can see how to figure out how long each student was in class for. We need to diff the rows so that we find out how long each student was clocked in for each session.

In [5]:
student_log.diff().head(10)

,student_id,timestamp
5,NaN,NaN
13,0,3.732698
0,0,2.972302
15,0,2.651204
4,0,1.014790
6,0,8.658445
3,1,-3.477183
11,0,0.584577
8,1,-15.899637
10,0,3.905739


Oops, we have a big jump when we switch students. We don't really want to do this, nor do we want to diff the student ID.

In [6]:
student_log['diff'] = student_log.groupby('student_id').diff(periods=1)
student_log.groupby('student_id').sum('diff')

,timestamp,diff
student_id,,
0,106.791871,19.029438
1,50.888266,0.584577
2,205.601924,23.398129
3,36.036400,11.808863


So what we are doing here is `grouping by` the `student_id`, then summing. This is like saying:

> For each student, sum the differences.

However, this is still wrong! We are summing All rows, when we should be summing every other row! This may be easier if we add `in` and `out` tags.

First we need to make a new column called `action` and we will give every row a value of `In`

In [7]:
# Make new column called action, assign every value 'In'
student_log['action'] = 'In'



Next we need to give every other row a value `Out`. I an going to use `.iloc` for this setp. `iloc` goes through the dataframe from top to bottom.
The slice command I give tells pandas to start at row 2 (id as 1 because 0 index) and skip every other row. 

In [8]:
# Switch every other column to say 'Out'. 
# iloc ignores indexes and goes through the dataframe in order from top to bottom.
student_log.iloc[1::2,-1] = 'Out'

# print results
student_log

,student_id,timestamp,diff,action
5,0,9.599590,NaN,In
13,0,13.332287,3.732698,Out
0,0,16.304589,2.972302,In
15,0,18.955793,2.651204,Out
4,0,19.970583,1.014790,In
6,0,28.629028,8.658445,Out
3,1,25.151845,NaN,In
11,1,25.736422,0.584577,Out
8,2,9.836785,NaN,In
10,2,13.742524,3.905739,Out


So the sum of the time in class should be the sum of the `Out` rows for each students. To get this we can either use `iloc` again, or we can use filtering.

In [9]:
# filter rows that are Out rows and sum the differences here only
student_log[student_log['action'] == 'Out'].groupby('student_id').sum('diff')

,timestamp,diff
student_id,,
0,60.917109,15.042346
1,25.736422,0.584577
2,112.799083,19.996242
3,23.922631,11.808863


In [10]:
# or simply
student_log[student_log['action'] == 'Out'].groupby('student_id').sum('diff')['diff']

student_id
0    15.042346
1     0.584577
2    19.996242
3    11.808863
Name: diff, dtype: float64

Perfect!

This is the same as saying:

> For each student, sum the difference of the out rows

## Conclusion

This gives the results we want. We can see who was in class for how long. I have made some assumptions here. That there are always in/out pairs and that there are no double in records caused by funny networking issues. In a future post I might think about addressing these other issues.

### Summary
we have used pandas functions to calculate how many minutes each student was in class. We can now see who we should mark as absent. Thanks for reading and let me know if you have any comments!